In [ ]:
# Goal: Make a multiprocessing pool, ensure each worker has persistent state.
from multiprocessing import cpu_count, Queue, Process, current_process
import time
import random

In [ ]:
STOP = 'STOP'

# From: "An example showing how to use queues to feed tasks to a collection of worker processes and collect the results:"
def worker(inputs, outputs):
    count = 0
    for value in iter(inputs.get, STOP):
        time.sleep(random.uniform(0.05, 0.1))
        count += 1
        outputs.put((count, value, current_process().name))
    print("DONE")

inputs = Queue()
map(inputs.put, range(100))
outputs = Queue()

ps = []
for i in range(10):
    p = Process(target=worker, args=(inputs, outputs))
    ps.append(p)
    p.start()
for p in ps:
    inputs.put(STOP)
for p in ps:
    p.join()

while not outputs.empty():
    print(outputs.get())
assert outputs.empty()

In [ ]:
def pooled_work(worker, values, process_count=None):
    inputs = Queue()
    map(inputs.put, values)
    outputs = Queue()
    # Need a more creative token.
    stop = (('__private_stop__', None,),)

    def target(inputs, outputs):
        values_iter = iter(inputs.get, stop)
        output_iter = worker(values_iter)
        for output in output_iter:
            outputs.put(output)

    if process_count is None:
        process_count = cpu_count()
    ps = []
    for i in xrange(process_count):
        p = Process(target=target, args=(inputs, outputs))
        ps.append(p)
        p.start()
    # Join, effectively flushing queue, possibly unordered.
    for p in ps:
        inputs.put(stop)
    for p in ps:
        p.join()
    # Since the pool is joined, queue should be flushed.
    out = []
    while not outputs.empty():
        out.append(outputs.get())
    return out

In [ ]:
# From: "An example showing how to use queues to feed tasks to a collection of worker processes and collect the results:"
def worker(values):
    # Count how much work each worker does.
    count = 0
    for value in values:
        time.sleep(random.uniform(0.05, 0.1))
        count += 1
        yield (current_process().name, count, value)

pooled_work(worker, range(5), process_count=3)